In [1]:
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
import numpy as np
import glob
from skimage.feature import hog

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import glob
import time
import os
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from collections import deque
## Import everything from helpers
from helpers.utils import Utils
from helpers.filterFalsePositive import FilterFalsePositive
from helpers.classify import ClassifierUtil
from helpers.vehicleDetector import VehicleDetector
from helpers.calibration import Camera
from helpers.warp import PerspectiveTransform
from helpers.threshold import ThresholdUtil
from helpers.line import Line
from helpers.lane import LaneSearch
% matplotlib inline


/Users/ankitgrover/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ankitgrover/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
## Load classifiers from pkl files
from sklearn.externals import joblib
svc = joblib.load('svcClassifier.pkl')
X_scaler = joblib.load('xScaler.pkl')


In [3]:
## Actual Parameters are used to identify cars from the images.
y_start = 400
y_stop = 656
scale = 1.8
orient = 12
pix_per_cell = 8
cell_per_block = 4
spatial_size = (32, 32)
hist_bins = 128
## Queue to store previous heatmaps and then club them together in last image frame
## It helps in keeping car always under the heat blogs and removing false positives.
heat_images = deque(maxlen = 3)
utils = Utils()
vehicle_detector = VehicleDetector()

In [4]:
## Detect vehicles using VehicleDetector class and find_cars funtion. 
## It is using HOG Subsampling approach internally to find cars.
def find_cars(image):
    return vehicle_detector.find_cars(image, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, window = 64)   


In [5]:
## Filter False positives using Heatmap and threshold.
def filter_false_positives(image, bboxes_list):
    filter_false_positive = FilterFalsePositive()
    heat = filter_false_positive.convert_img_to_heat(image)
    #print (bboxes)
    heatmap = filter_false_positive.add_heat(bboxes_list)
    ## Averaging last 3 heatmaps to smoothen the image frames output. 
    heat_images.append(heatmap)
    heatmap = np.sum(np.array(heat_images), axis = 0)
    heatmap = filter_false_positive.threshold(threshold = 5)
    return heatmap, filter_false_positive

In [6]:
## Draw labels on the image frame after removing false positives.
## After thresholding only blobs of heat left around the cars positions.
def labels(image, filter_false_positive):
    filter_false_positive.labels()
    labeled_img = filter_false_positive.draw_labeled_bboxes(image)
    return labeled_img

In [7]:
#### Lane Detection

## Camera Calibration
## Load camera calibration images
def load_camera_calibration_images(folder='camera_cal'):
    camera_files = os.listdir(folder)
    paths = list(map(lambda f: os.path.join(folder,f), camera_files))
    print (paths)
    cal_images = [mpimg.imread(path) for path in paths]
    return cal_images

## Load images from directory
def load_img_from_dir(folder, filename):
    path = os.path.join(folder, filename)
    return mpimg.imread(path)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [8]:
## Load camera images and calibrate camera using Camera class
images = load_camera_calibration_images()
camera = Camera()
mtx, dist = camera.calibrate(images)

['camera_cal/calibration1.jpg', 'camera_cal/calibration10.jpg', 'camera_cal/calibration11.jpg', 'camera_cal/calibration12.jpg', 'camera_cal/calibration13.jpg', 'camera_cal/calibration14.jpg', 'camera_cal/calibration15.jpg', 'camera_cal/calibration16.jpg', 'camera_cal/calibration17.jpg', 'camera_cal/calibration18.jpg', 'camera_cal/calibration19.jpg', 'camera_cal/calibration2.jpg', 'camera_cal/calibration20.jpg', 'camera_cal/calibration3.jpg', 'camera_cal/calibration4.jpg', 'camera_cal/calibration5.jpg', 'camera_cal/calibration6.jpg', 'camera_cal/calibration7.jpg', 'camera_cal/calibration8.jpg', 'camera_cal/calibration9.jpg']


In [9]:
## Define parmams that needs to be initialized before computing lanes in any input video.
threshold_util = None
lane_search = None
perspective_transform = None
index = 0

In [10]:
def init_params():
    ## Initialize the objects before running images on the pipeline.
    global index
    global threshold_util 
    global lane_search
    global perspective_transform
    threshold_util = ThresholdUtil()
    lane_search = LaneSearch()
    perspective_transform = PerspectiveTransform()
    index = 0

In [15]:
''' Process the input image with the pipeline (Undistort Image -> 
Apply Combined Threshold technques: HLS (Only Saturation), Sobelx, Sobely, Magnitude and Directional Gradient -> 
Warp Image -> 
Histogram to identify the peaks ->
Sliding Window or Look Ahead Filter to mask the peaks with polynomial fits ->
Average the polynomial fit coefficients to smoothen the fits ->

Sanity Check with Radius of Curvature)
Pipeline for Vehicle Detection also (Find Cars -> Heatmap-> Thresholding Heatmap -> Labels the Cars)
'''
def process_image(image):
    ## Lane Detection Pipeline.
    global index
    ## Undistort image.
    undistort_img = camera.undistort(image)
    index +=1
    ## Create combined HLS (Saturation) color, sobel, magnitude and directional threshold image.
    threshold_image = threshold_util.create_combined_color_thresh(undistort_img)
    ## Warp the image using Perspective transform bird view.
    warp_img = perspective_transform.warp(threshold_image)
    ##cv2.imwrite("output_images/Warp" + str(index) + ".jpg",warp_img)
    ## Search for a lane with confident coefficients or do searching first time
    if (lane_search.confident):
        ## Search for a lane with look ahead filter
        draw_image = lane_search.sliding_window_look_ahead_filter(warp_img)
    else:
        ## Search for a lane and draw the results on the image   
        draw_image = lane_search.sliding_window(warp_img)
    undist_img_with_results = lane_search.draw_lane_results(warp_img, perspective_transform.Minv, undistort_img)
    ### Vehicle Detection Pipeline
    draw_img, bboxes = find_cars(undist_img_with_results)
    heatmap, filter_false_positive = filter_false_positives(undist_img_with_results, bboxes)
    undist_img_with_results = labels(undist_img_with_results, filter_false_positive)
    return undist_img_with_results

In [16]:
## Read Project video and ouput video with labels of Cars.
from moviepy.editor import VideoFileClip
init_params()
output_video_path = 'output_video/project_video_lane_vehicle.mp4'
input_clip = VideoFileClip("project_video.mp4")
output_clip = input_clip.fl_image(process_image)
%time output_clip.write_videofile(output_video_path, audio=False)

[MoviePy] >>>> Building video output_video/project_video_lane_vehicle.mp4
[MoviePy] Writing video output_video/project_video_lane_vehicle.mp4


100%|█████████▉| 1260/1261 [12:52<00:00,  1.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video/project_video_lane_vehicle.mp4 

CPU times: user 12min 5s, sys: 1min 53s, total: 13min 59s
Wall time: 12min 53s
